In [2]:
import cv2 as cv
import os
import numpy as np
import json

In [3]:
import os
from scipy.ndimage import gaussian_filter
import numpy as np 
import pandas as pd 
import time 
from PIL import Image
import cv2
import pickle 
import matplotlib.pyplot as plt 
from skimage.feature import hog

In [4]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [5]:
path_dir = os.getcwd()
path_dir

'i:\\WPI\\Fall 2020\\Computer Vision - RBE 549\\CV Project\\Dataset\\cars_train\\Dimensionality Experiments'

In [6]:
fold_dir = '\\'.join((path_dir, "Images"))
car_names = os.listdir(fold_dir)

In [7]:
#Without Resizing:
start_time = time.time()

img_cars = []
for file_n in car_names:
    file_name = "\\".join((fold_dir, file_n))
    img = np.array(Image.open(file_name).convert('LA'))
    #img_grey = tf.image.resize(img, (256,256),preserve_aspect_ratio=False).numpy()
    img_cars.append(img[:,:,0])

print(time.time() - start_time)

1.3873605728149414


In [8]:
#For Image Resizing:
start_time = time.time()

img_cars_resized = []
for file_n in car_names:
    file_name = "\\".join((fold_dir, file_n))
    img = np.array(Image.open(file_name).convert('LA'))
    img_grey = cv2.resize(img, (256,256)) 
    img_cars_resized.append(img_grey[:,:,0])

print(time.time() - start_time)

1.429199457168579


In [15]:
start_time = time.time()
freak = cv2.xfeatures2d.FREAK_create()
desc_freak_normal = []
kp_normal = []
for img in img_cars:
    img = cv2.bilateralFilter(img, 9, 75, 75)
    fast = cv2.FastFeatureDetector_create(12)
    kp = fast.detect(img, None)
    kp, des = freak.compute(img, kp)

    if not des is None:   
        desc_freak_normal.append(des)
    # if not kp is None:
    #     kp_normal.append(kp)

print(time.time() - start_time)

1.4561100006103516


In [14]:
start_time = time.time()
freak = cv2.xfeatures2d.FREAK_create()
desc_freak_scaled = []
kp_scaled = []
for img in img_cars_resized:
    img = cv2.bilateralFilter(img, 9, 75, 75)
    fast = cv2.FastFeatureDetector_create(12)
    kp = fast.detect(img, None)
    kp, des = freak.compute(img, kp)

    if not des is None:   
        desc_freak_scaled.append(des)
    # if not kp is None:
    #     kp_normal.append(kp)


print(time.time() - start_time)

0.39896368980407715


In [20]:
flatten_scaled = []
scaled_shape = []
flatten_scaled_shape = []
for desc in desc_freak_scaled:
    flatten_scaled.append(np.resize(desc, (-1)))
    scaled_shape.append(np.shape(desc)[0])
    flatten_scaled_shape.append(np.shape(np.resize(desc, (-1))))

In [30]:
#Typical Descriptor for Original Image
print(np.shape(desc_freak_normal[0]))
print(np.shape(desc_freak_normal[1]))
print(np.shape(desc_freak_normal[2]))
print(np.shape(desc_freak_normal[3]))
print(np.shape(desc_freak_normal[4]))

(412, 64)
(1668, 64)
(1024, 64)
(621, 64)
(111, 64)


In [31]:
#Typical Descriptor for Scaled Images
print(np.shape(desc_freak_scaled[0]))
print(np.shape(desc_freak_scaled[1]))
print(np.shape(desc_freak_scaled[2]))
print(np.shape(desc_freak_scaled[3]))
print(np.shape(desc_freak_scaled[4]))

(211, 64)
(527, 64)
(300, 64)
(148, 64)
(151, 64)


In [18]:
flatten_original = []
normal_shape = []
flatten_normal_shape = []
for desc in desc_freak_normal:
    flatten_original.append(np.resize(desc, (-1)))
    normal_shape.append(np.shape(np.resize(desc, (-1))))
    flatten_normal_shape.append(np.shape(np.resize(desc, (-1))))

In [21]:
count, values = np.unique(normal_shape, return_counts=True)
print("Maximum Number of Descriptors - Original: ", np.max(normal_shape))
print("Minimum Number of Descriptors - Original: ",np.min(normal_shape))
print()
print(count)
print(values)

Maximum Number of Descriptors - Original:  470911
Minimum Number of Descriptors - Original:  511

[   511   7103   8383   9919  12031  13183  13503  14271  14719  17023
  17535  17983  18431  18751  20095  20671  23807  23935  25087  25151
  26367  28031  28735  29375  29567  29631  30655  30719  32127  32255
  32511  32639  33023  35455  35647  35839  36543  36991  37759  38719
  38783  38911  38975  39295  39743  41023  42367  42943  43263  46527
  47359  48127  48191  48255  49407  49983  51263  53375  54527  55935
  56639  56703  57407  60351  63487  64575  65535  65983  67263  68927
  70975  71487  73471  74943  75135  76863  79615  79935  81151  87167
  87487  88575  90815  93119  98111 100223 100351 101119 106751 108287
 110911 112319 115135 122879 131583 146239 276415 307391 440191 470911]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

In [22]:
count, values = np.unique(scaled_shape, return_counts=True)
print("Maximum Number of Descriptors - Scaled: ", np.max(scaled_shape))
print("Minimum Number of Descriptors - Scaled: ", np.min(scaled_shape))
print()
print(count)
print(values)

Maximum Number of Descriptors - Scaled:  790
Minimum Number of Descriptors - Scaled:  72

[ 72 123 133 136 148 151 155 161 164 167 174 176 178 179 180 182 183 188
 189 206 208 211 213 223 226 228 229 232 233 245 248 250 252 259 270 273
 282 288 291 296 300 305 306 308 309 311 313 314 315 323 329 332 334 335
 341 342 343 348 358 364 367 369 372 373 376 382 389 402 405 414 421 429
 439 443 448 455 462 479 488 490 507 514 516 523 527 560 566 567 627 761
 790]
[1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 2
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [23]:
count, values = np.unique(flatten_scaled_shape, return_counts=True)
print("Maximum Number of Features - Scaled-Flatten: ", np.max(flatten_scaled_shape))
print("Minimum Number of Features - Scaled-Flatten: ", np.min(flatten_scaled_shape))
print()
print(count)
print(values)

Maximum Number of Features - Scaled-Flatten:  50559
Minimum Number of Features - Scaled-Flatten:  4607

[ 4607  7871  8511  8703  9471  9663  9919 10303 10495 10687 11135 11263
 11391 11455 11519 11647 11711 12031 12095 13183 13311 13503 13631 14271
 14463 14591 14655 14847 14911 15679 15871 15999 16127 16575 17279 17471
 18047 18431 18623 18943 19199 19519 19583 19711 19775 19903 20031 20095
 20159 20671 21055 21247 21375 21439 21823 21887 21951 22271 22911 23295
 23487 23615 23807 23871 24063 24447 24895 25727 25919 26495 26943 27455
 28095 28351 28671 29119 29567 30655 31231 31359 32447 32895 33023 33471
 33727 35839 36223 36287 40127 48703 50559]
[1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 2
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [24]:
count, values = np.unique(flatten_normal_shape, return_counts=True)
print("Maximum Number of Features - Original-Flatten: ",np.max(flatten_normal_shape))
print("Minimum Number of Features - Original-Flatten: ",np.min(flatten_normal_shape))
print()
print(count)
print(values)

Maximum Number of Features - Original-Flatten:  470911
Minimum Number of Features - Original-Flatten:  511

[   511   7103   8383   9919  12031  13183  13503  14271  14719  17023
  17535  17983  18431  18751  20095  20671  23807  23935  25087  25151
  26367  28031  28735  29375  29567  29631  30655  30719  32127  32255
  32511  32639  33023  35455  35647  35839  36543  36991  37759  38719
  38783  38911  38975  39295  39743  41023  42367  42943  43263  46527
  47359  48127  48191  48255  49407  49983  51263  53375  54527  55935
  56639  56703  57407  60351  63487  64575  65535  65983  67263  68927
  70975  71487  73471  74943  75135  76863  79615  79935  81151  87167
  87487  88575  90815  93119  98111 100223 100351 101119 106751 108287
 110911 112319 115135 122879 131583 146239 276415 307391 440191 470911]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 